<a href="https://colab.research.google.com/github/snikhil17/Wikipedia_Data_Analysis/blob/main/Wikipedia_Climate_Change_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import wikipedia

ModuleNotFoundError: ignored

In [ ]:
!ls

sample_data
